In [17]:
%%html
<style>
.container {width: 99% !important; }
#logo
{
    float: left;
}
#names
{
    float: right;
    
}
#logo2
{
    float: right;
    height=100;
    width=100;
    margin: 5px;
}
#top_menu
{
    margin: 20px auto 0;
    
    height:300px;
    width: 400px;
}
</style>




In [18]:
from arcgis.gis import GIS

In [ ]:
gis = GIS(username='AndrewSolutions')


# Mapping Flight Paths

<center><img src="./img/airplane-fact.jpg" width=500/></center>

In [19]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("./data/flights.db")
df_airlines = pd.read_sql_query("SELECT * from airlines", conn)
df_airlines.head()

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [20]:
df_routes = pd.read_sql_query("select * from routes", conn)
df_routes.head()

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


In [21]:
df_apt = pd.read_sql_query("select * from airports", conn)
df_apt.head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


## Data Wrangling

- **Type Issues** latitude/longitude come back as strings

In [22]:
df_apt.latitude = df_apt.latitude.astype(float)
df_apt.longitude = df_apt.longitude.astype(float)

In [23]:
from arcgis.geometry import Point
sr = {'wkid' : 4326}
geoms = [Point({'x': i[2], 'y':i[1], 'spatialReference' : sr}) \
        for i in df_apt[['latitude', 'longitude']].to_records()]

In [24]:
from arcgis.features import SpatialDataFrame
sdf = SpatialDataFrame(data=df_apt, geometry=geoms)
m = GIS().map('Brazil')
m

In [25]:
sdf.plot(kind='map', 
         map_widget=m,
         marker_size=3, 
         pallette='Blues_r',
         symbol_style='o',
         outline_color='Reds_r',
         line_width=.5,
         cstep=25)

In [ ]:
from arcgis.geometry import Polyline
import greatcircle, sys
import numpy as np

## Plotting the Routes

- Get the source airport for each route
- Get the destination airport for each route
- Cast to floats
- Plot on a map

In [26]:
routes = pd.read_sql_query("""
                           select cast(sa.longitude as float) as source_lon, 
                           cast(sa.latitude as float) as source_lat,
                           cast(da.longitude as float) as dest_lon,
                           cast(da.latitude as float) as dest_lat
                           from routes 
                           inner join airports sa on
                           sa.id = routes.source_id
                           inner join airports da on
                           da.id = routes.dest_id;
                           """, 
                           conn)
routes.head()

,source_lon,source_lat,dest_lon,dest_lat
0,39.956589,43.449928,49.278728,55.606186
1,48.006278,46.283333,49.278728,55.606186
2,48.006278,46.283333,43.081889,44.225072
3,61.503333,55.305836,49.278728,55.606186
4,61.503333,55.305836,82.650656,55.012622


In [16]:
gcs = []
recs = []
for name, row in routes[:3000].iterrows():
    if abs(row["source_lon"] - row["dest_lon"]) < 90:
        # Draw a great circle between source and dest airports.
        recs.append(row.tolist())
        gc = greatcircle.GreatCircle(lat1=row['source_lat'],
                               lat2=row['dest_lat'],
                               lon1=row['source_lon'],
                               lon2=row['dest_lon'])
        lat, long = gc.points(5000)
        gcs.append(
        Polyline({
            'paths' : [np.column_stack([list(lat), list(long)]).tolist()],
            'spatialReference' : {'wkid' : 4326}
        }))

In [27]:
routes_sdf = SpatialDataFrame(data=recs, geometry=gcs, columns=routes.columns, sr=4326)
routes_sdf.head()

,source_lon,source_lat,dest_lon,dest_lat,SHAPE
0,39.956589,43.449928,49.278728,55.606186,"{'paths': [[[39.956589, 43.449928], [39.958047..."
1,48.006278,46.283333,49.278728,55.606186,"{'paths': [[[48.00627800000001, 46.283333], [4..."
2,48.006278,46.283333,43.081889,44.225072,"{'paths': [[[48.00627800000001, 46.283333], [4..."
3,61.503333,55.305836,49.278728,55.606186,"{'paths': [[[61.503333000000005, 55.305836], [..."
4,61.503333,55.305836,82.650656,55.012622,"{'paths': [[[61.503333000000005, 55.305836], [..."


In [28]:
m = GIS().map('Europe')
m

In [29]:
sdf.plot(kind='map', 
         map_widget=m,
         marker_size=3, 
         pallette='Blues_r',
         symbol_style='o',
         outline_color='Greens_r',
         line_width=.5,
         cstep=25)

In [30]:
routes_sdf.head(500).copy().plot(kind='map', 
                                 map_widget=m,
                                line_width=.5,
                                pallette='Reds_r',
                                cstep=50)